In [ ]:
Prophet은 페이스북에서 공개한 시계열 예측 라이브러리 인데요, 정확도가 높고 빠르며 직관적인 파라미터로 모델 수정이 용이하다는 장점을 갖고 있음
Prophet 모델의 주요 구성요소는 Trend, Seasonality, Holiday 입니다. 이 세가지를 결합하면 아래의 공식으로 나타낼 수 있습니다.
y(t)=g(t)+s(t)+h(t)+ϵi

g(t) : piecewise linear or logistic growth curve for modelling non-periodic changes in time series
s(t) : periodic changes (e.g. weekly/yearly seasonality)
h(t) : effects of holidays (user provided) with irregular schedules
ϵi: error term accounts for any unusual changes not accommodated by the model

위에서 Trend 를 구성하는 g(t) 함수는 주기적이지 않은 변화인 트렌드를 나타냅니다. 부분적으로 선형 또는 logistic 곡선으로 이루어져 있습니다. 그리고 Seasonality 인 
s(t) 함수는 weekly, yearly 등 주기적으로 나타나는 패턴들을 포함합니다.
Holiday를 나타내는 h(t) 함수는 휴일과 같이 불규칙한 이벤트들을 나타냅니다. 만약 특정 기간에 값이 비정상적으로 증가 또는, 감소했다면, holiday로 정의하여 모델에 반영할 수 있습니다. 마지막으로 
ϵi 는 정규분포라고 가정한 오차입니다.






In [ ]:
https://hyperconnect.github.io/2020/03/09/prophet-package.html

In [4]:
import numpy as np
import pandas as pd
import time
import glob
import pickle
import itertools

import seaborn as sns
import matplotlib.pyplot as plt  # from matplotlib import pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

# !pip install pystan
# !pip install fbprophet

from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
from fbprophet.plot import add_changepoints_to_plot
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import ParameterGrid

import warnings
warnings.filterwarnings('ignore')

# 데이터프레임 출력 옵션
pd.set_option('display.max_columns', 100)

#지수표현
pd.options.display.float_format = '{:.5f}'.format

# # 그래프 폰트
# font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
# rc('font', family=font_name)
# plt.rc('font', family='Malgun Gothic')
# plt.rcParams["figure.figsize"] = (8, 4)
# plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# all_params
li_copy = df_summ[~df_summ[['SIDO_EDU_NM', 'PART_EDU_NM', 'SCHOOL_TP']].duplicated()][['SIDO_EDU_NM', 'PART_EDU_NM','SCHOOL_TP']]
li_copy = zip(li_copy.SIDO_EDU_NM, li_copy.PART_EDU_NM, li_copy.SCHOOL_TP)
# 시도별 prophet
li_sch = df_summ.SCHOOL_TP.unique()
li_sido = df_summ.SIDO_EDU_NM.unique()
df_pred = pd.DataFrame()
df_result = pd.DataFrame()

model_parameters = pd.DataFrame()

for s,sg,d in tqdm(li_copy):
    df_hap = df_summ[(df_summ.SIDO_EDU_NM == s) & (df_summ.PART_EDU_NM == sg) & (df_summ.SCHOOL_TP == d)]
    df_hap21 = df_summ21[(df_summ21.SIDO_EDU_NM == s) & (df_summ21.PART_EDU_NM == sg) & (df_summ21.SCHOOL_TP == d)]
    print("*** %s %s %s ***" % (s, sg, d))
        
            
##############################################################           

# 모델 적합
df_prophet = Prophet(changepoint_prior_scale = 0.15, daily_seasonality = True)
df_prophet.fit(df)

# 향후 1년간의 time stamp 생성
예측을 위해서는 예측하고자 하는 날짜가 ds 컬럼에 있어야함
Prophet.make_future_dataframe을 사용하여 지정된 날짜 수만큼 미래로 확장되는 적절한 데이터 프레임을 얻을 수 있음
기본적으로 모형의 훈련에 사용된 시계열의 날짜도 포함되므로 모델이 맞는지 확인할 수 있음

fcast_time = 90
df_forecast = df_prophet.make_future_dataframe(periods = fcast_time, freq = 'D')
df_forecast.tail(10)
df_forecast = df_prophet.predict(df_forecast)

# 예측 결과 확인
df_forecast[['ds','yhat',..]].tail()

# 예측값 시각화
df_prophet.plot(df_forecast, xlabel = 'Data', ylabel = 'Price($)')

## 예측 구성요소 확인
# 예측에 사용된 구성 요소는 Prophet.plot_components 메소드를 사용하여 확인 가능
# 기본적으로 시계열의 추세, 연간 계절성, 주간 계절성이 표시됨
df_prophet.plot_components(df_forecast)
plt.show()

# 교차검증
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(df_prophet, initial = '1095 days', period = '180 days', horizon = '365 days')

# 모형 성능 확인
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head()

# 교차검증 결과 시각화
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric = 'mae')

##############################################################           
            
    # prophet
    model = Prophet(yearly_seasonality=10, weekly_seasonality=False, daily_seasonality=False, 
                    changepoint_range=1, changepoint_prior_scale=0.1, n_changepoints=7)
    
#     # 모델의 Trend를 조절할 수 있는 파라미터는 다음과 같습니다.
#     changepoints : 트렌드 변화시점을 명시한 리스트값
#     changepoint_prior_scale : changepoint(trend) 의 유연성 조절
#     n_changepoints : changepoint 의 개수
#     changepoint_range : changepoint 설정 가능 범위. (기본적으로 데이터 중 80% 범위 내에서 changepoint를 설정합니다.)
    
    
# 모델의 changepoint 를 시각화해보고, changepoint_prior_scale 값 변경에 따른 Trend 변화를 살펴보겠습니다.
# changepoint_prior_scale = 0.05 (default)
# 빨간 실선은 트렌드를 의미하며, 빨간 점선은 트렌드가 변화하는 changepoint 를 의미
from fbprophet.plot import add_changepoints_to_plot

fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

# changepoint_prior_scale = 0.3 인 경우
m = Prophet(changepoint_prior_scale=0.3)
m.fit(df)

fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

# changepoint_prior_scale 값을 0.3으로 높여준 후 트렌드를 더 유연하게 감지하는 것을 확인할 수 있습니다.
# 이 값을 너무 높여버리면 overfitting의 위험이 있으니 주의해야 합니다.
# 만약 트렌드가 바뀌는 시점(서비스 확대 배포 또는 프로모션 등으로 인한 변화 시점)을 알고 있다면, changepoints 파라미터를 추가할 수 있고, changepoints 수 또한 n_changepoints 로 지정할 수 있습니다. 물론, 이 두 파라미터를 설정해주지 않아도 모델이 자동으로 감지합니다.




    
    
#     prophet = Prophet()
    model.fit(df_hap[['ds', 'y']])
                      
    # 5년 예측
    future = model.make_future_dataframe(periods=5, freq='y')  #  periods 값은 향후 몇일 (또는 주,월 등 단위 주기) 을 예측할 것인지를 의미
    forecast = model.predict(future)

    # 예측값 시각화
    fig1 = model.plot(forecast)
    
    # 피팅된 모델의 컴포넌트들을 시각화
    fig2 = model.plot_components(forecast)
    
    # 컴포넌트 시각화 해석
    # => 만약 모델이 데이터의 Trend를 잘 잡아내지 못하는 것 같다면, changepoint_prior_scale 파라미터값을 높여주어 changepoint를 더 민감하게 감지하도록 할 수 있습니다. 여기서 changepoint란, Trend가 변화하는 지점을 의미
    # => 아래 차트는 각각 ‘주 계절성’과 ‘연 계절성’을 의미
    # => Trend와 마찬가지로 Seasonality또한 seasonality_prior_scale 파라미터로 모델 반영 강도를 조절할 수 있습니다.
    
    
    
    
    
    
    
    
    
    
    
    
    # 학생 수는 음의 값이 나올 수 없기 때문에 -인 경우 0으로 변경
    forecast.yhat = forecast.yhat.apply(lambda x: 0 if x <= 0 else x)
    
    fore = forecast.loc[:, ['ds', 'yhat']]
    fore.columns = ['ds', 'PRED_VAL']
#     fore_df = fore[fore.ds <= '2020-12-31']
    result2 = fore.copy(deep=True)
    
    # 실제값이랑 예측값 합치기
    df_real = df_hap21[df_hap21['SIDO_EDU_NM'] == s]
    df_real = df_real[['BASE_YY', 'y']]
    df_real.columns = ['BASE_YY', 'REAL_VAL']
    
    result2['BASE_YY'] = result2['ds'].dt.year.astype('str')
    result2 = result2.drop('ds', axis=1)
    
    df_pred_real = result2.merge(df_real, on='BASE_YY', how='outer')
    df_pred_real['시도교육청'] = s
    df_pred_real['교육지원청'] = sg
    df_pred_real['학교구분'] = d
    
    df_pred_real = df_pred_real[['BASE_YY','시도교육청','교육지원청','학교구분','REAL_VAL','PRED_VAL']]
    df_result = df_result.append(df_pred_real)
    df_result['PRED_VAL'] = np.where(df_result['PRED_VAL'] < 0, 0, df_result['PRED_VAL'])
#     result2['BASE_YY'] = pd.DatetimeIndex(result2['ds']).year.astype('str')
#     result2 = result2.drop(['ds'], axis=1)
    
    # 정확도
    df_result['acc'] = round(
    1 - abs((df_result['REAL_VAL'] - df_result['PRED_VAL']) / df_result['REAL_VAL'])) * 100

    
    # 그래프
    title = '%s %s %s' % (s,sg,d)
     # plt = get_prophet_plot(prophet, forecast, title)
    plt = get_line_plot(df_pred_real, 'BASE_YY', ['REAL_VAL', 'PRED_VAL'])
    plt.title(title, size=20)
    plt.savefig(res_path + 'prophet_%s.png' % (title))
    plt.close()
    
    df_sub_acc = df_pred_real[~df_pred_real['BASE_YY'].isin(['2021', '2022', '2023', '2024', '2025'])]
    r2 = r2_score(df_sub_acc['REAL_VAL'], df_sub_acc['PRED_VAL'])
            
    se = np.square(df_result['REAL_VAL'] - df_result['PRED_VAL'])
    mse = np.mean(se)
    rmse = np.sqrt(mse)
    df_result['mse'] = se
        # print('Mean Squared Error(MSE)--------', MSE)
        
    # 오류는 안나는데 데이터가 비정상적으로 많이 출력
#     model_parameters = model_parameters.append(pd.DataFrame({'시도': s, '교육지원청명':sg , '학교구분': d ,'결정계수':r2,'MSE' : mse}, index=[0]))

            

In [ ]:
result2['BASE_YY'] = result2['ds'].dt.year.astype('str')
result2 = result2.drop('ds', axis=1)
result2